In [2]:
import pandas as pd
import numpy as np
import random as rd

path = '/Users/Rizal Maulana/Downloads/semester 7/Alev/'

In [3]:
df_node = pd.read_excel(path+'Data Jarak.xlsx',index_col=0,header=1)
mapping = pd.Series(df_node.index)
df_node.index = range(len(df_node.index))
df_node.columns = range(len(df_node.columns))
df_node

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,7.6,6.3,11.3,1.5,3.8,2.1,7.0,4.8,10.9,6.7,9.5,12.2,5.6,6.3,11.3
1,7.6,0.0,10.9,12.8,6.9,7.7,6.4,8.3,7.4,10.6,2.3,7.5,13.8,6.5,1.9,13.0
2,6.3,10.9,0.0,8.8,7.5,4.2,7.4,5.1,3.7,8.4,9.9,8.3,9.7,5.5,10.1,7.9
3,11.3,12.8,8.8,0.0,13.7,8.2,12.3,6.2,7.2,6.4,11.5,7.4,4.7,8.2,12.6,2.8
4,1.5,6.9,7.5,13.7,0.0,4.9,1.6,8.2,6.0,12.1,5.7,9.7,13.4,5.6,5.4,12.4
5,3.8,7.7,4.2,8.2,4.9,0.0,4.3,3.8,1.4,7.7,6.8,6.4,9.0,1.4,6.9,8.2
6,2.1,6.4,7.4,12.3,1.6,4.3,0.0,7.3,5.2,11.3,5.1,7.5,13.3,4.2,5.1,11.6
7,7.0,8.3,5.1,6.2,8.2,3.8,7.3,0.0,3.0,4.3,6.7,4.6,5.6,3.8,7.8,5.3
8,4.8,7.4,3.7,7.2,6.0,1.4,5.2,3.0,0.0,6.5,6.5,5.6,7.8,2.1,6.7,6.8
9,10.9,10.6,8.4,6.4,12.1,7.7,11.3,4.3,6.5,0.0,8.4,3.3,7.3,7.7,9.9,9.3


# M-TSP

In [4]:
class MTSP(object):
    def __init__(self,df_node,awal,flag,populasi=10,generasi=1):
        self.df_node = df_node
        self.awal = awal
        self.flag = flag
        self.populasi = populasi
        self.generasi = generasi
        self.df_populasi = pd.DataFrame(0,index=range(0,populasi),columns=['Kromosome','Cost','Fitness'])
        
    def main(self,cr,mr):
        krom,cost,fitness = [],[],[]
        for loop in range(self.populasi):
            kromosome = rd.sample([i for i in self.df_node.index if i != self.awal],len(self.df_node.index)-1)
            cost_temp = self.total_cost(kromosome)
            fitnes_temp = self.total_fitness(cost_temp)
            krom.append(kromosome)
            cost.append(cost_temp)
            fitness.append(fitnes_temp)

        self.df_populasi['Kromosome'] = krom
        self.df_populasi['Cost'] = cost
        self.df_populasi['Fitness'] = fitness
        
        for loop in range(self.generasi):
            print('Generasi Ke-{}'.format(loop))
            self.reproduction(cr,mr)
            self.selection()
            print('Populasi Bertahan :')
            print(self.df_populasi.head())
#         print(self.df_populasi.head(1)['Fitness'],'\n')

    def reproduction(self,cr,mr):
        self.crossover(cr)
        self.mutation(mr)
        
        krom_ = pd.Series(self.child)
        cost_ = pd.Series([self.total_cost(i) for i in self.child])
        fitness_ = pd.Series([self.total_fitness(i) for i in cost_])
        df_child = pd.concat([krom_,cost_,fitness_],axis=1)
        df_child.columns = ['Kromosome','Cost','Fitness']
        self.df_c = self.df_populasi.copy(deep=True)
        self.df_c = pd.concat([self.df_c,df_child])
        self.df_c.index = range(len(self.df_c))
        
    def crossover(self,cr):
        jml_cros = round(cr*self.populasi)
        self.child = []

        for loop in range(jml_cros):
            cut = rd.randint(self.df_node.index[0],self.df_node.index[-2])
            parent = rd.sample([i for i in self.df_populasi.index],2)
    
            child_temp = [i for k,i in enumerate(self.df_populasi.loc[parent[0],'Kromosome']) if k <= cut]
            for i in self.df_populasi.loc[parent[1],'Kromosome']:
                if i not in child_temp:
                    child_temp.append(i)
            
            self.child.append(child_temp)
       
    def mutation(self,mr):
        jml_mut = round(mr*self.populasi)
        for i in range(jml_mut):
            cut = rd.sample([i for i in self.df_node.index[:-2]],2)
            parent = rd.randint(self.df_populasi.index[0],self.df_populasi.index[-1])

            child_temp = self.df_populasi.loc[parent,'Kromosome']
            child_temp[cut[0]], child_temp[cut[1]] = child_temp[cut[1]], child_temp[cut[0]]
            self.child.append(child_temp)
    
    def selection(self):
        self.df_populasi = self.df_c.nlargest(self.populasi,'Fitness')
        self.df_populasi.index = range(len(self.df_populasi))
        
    def total_cost(self,kromosome):
        jarak = [[self.awal]+kromosome[s:e+1]+[self.awal] for s,e in self.flag]
        tot_jarak = []
        cost = 0

        for i in jarak:
            awal = 0
            temp = []
            for j in i:
                temp.append(i[awal:awal+2])
                awal += 1
            tot_jarak += temp[:-1]

        for i in tot_jarak:
            cost += self.df_node.loc[i[0],i[1]]

        return cost

    def total_fitness(self,cost):
        return 1/cost

# Pengujian Jumlah cr dan mr

In [5]:
kombinasi = [(0.9,0.1),(0.8,0.2),(0.7,0.3),(0.6,0.4),(0.5,0.5),(0.4,0.6),(0.3,0.7),(0.2,0.8),(0.1,0.9)]
mtsp = MTSP(df_node,0,[(0,7),(8,14)],populasi=10,generasi=20)
for i in kombinasi:
    cr,mr=i
    print('Cr : {}, Mr : {}'.format(cr,mr))
    for loop in range(10):
        print('Percobaan Ke-{}'.format(loop))
        mtsp.main(cr,mr)

Cr : 0.9, Mr : 0.1
Percobaan Ke-0
Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [6, 11, 1, 14, 9, 12, 2, 5, 3, 7, 4, 13, 8, 10...  116.2  0.008606
1  [3, 9, 2, 5, 14, 1, 12, 6, 11, 10, 4, 13, 7, 8...  119.7  0.008354
2  [11, 14, 15, 9, 3, 7, 2, 6, 1, 5, 12, 8, 10, 4...  121.4  0.008237
3  [7, 8, 13, 10, 15, 14, 2, 4, 9, 5, 1, 12, 3, 1...  122.2  0.008183
4  [7, 8, 13, 10, 15, 14, 2, 4, 9, 5, 1, 12, 3, 1...  122.2  0.008183
Generasi Ke-1
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [6, 11, 1, 14, 9, 12, 2, 5, 3, 7, 4, 13, 8, 10...  116.2  0.008606
1  [6, 11, 1, 14, 9, 12, 2, 5, 3, 7, 4, 13, 8, 10...  116.2  0.008606
2  [3, 9, 2, 5, 14, 1, 12, 6, 11, 10, 4, 13, 7, 8...  119.7  0.008354
3  [3, 9, 2, 5, 14, 1, 12, 6, 11, 10, 4, 13, 7, 8...  119.7  0.008354
4  [11, 14, 15, 9, 3, 7, 2, 6, 1, 5, 12, 8, 10, 4...  121.4  0.008237
Generasi Ke-2
Populasi Bertahan :
                        

Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [6, 14, 1, 11, 3, 12, 9, 2, 5, 8, 4, 10, 13, 7...  93.6  0.010684
1  [6, 14, 1, 11, 3, 12, 9, 2, 5, 8, 4, 10, 13, 7...  93.6  0.010684
2  [6, 14, 1, 12, 3, 9, 11, 2, 8, 5, 4, 10, 13, 7...  94.7  0.010560
3  [6, 14, 1, 12, 3, 9, 11, 2, 5, 8, 4, 10, 13, 7...  94.8  0.010549
4  [6, 14, 1, 12, 3, 9, 11, 2, 5, 8, 13, 10, 4, 7...  94.8  0.010549
Percobaan Ke-1
Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [4, 11, 8, 12, 9, 14, 2, 5, 6, 13, 1, 10, 3, 1...  101.6  0.009843
1  [11, 9, 7, 10, 14, 15, 5, 4, 2, 6, 1, 13, 8, 1...  105.4  0.009488
2  [5, 2, 3, 12, 11, 1, 15, 7, 14, 10, 9, 4, 8, 1...  106.1  0.009425
3  [7, 10, 14, 15, 9, 5, 4, 2, 6, 11, 1, 13, 8, 1...  113.4  0.008818
4  [11, 9, 7, 8, 14, 2, 5, 4, 15, 13, 10, 6, 1, 1...  114.0  0.008772
Generasi Ke-1
Populasi Bertahan :
                                           Kromosome   Cost   F

Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [14, 9, 7, 10, 11, 3, 8, 5, 4, 1, 6, 13, 12, 2...  91.7  0.010905
1  [11, 9, 7, 10, 14, 2, 8, 4, 5, 6, 1, 13, 12, 3...  96.0  0.010417
2  [11, 9, 7, 10, 14, 2, 8, 4, 5, 6, 1, 13, 12, 3...  97.1  0.010299
3  [11, 9, 7, 10, 14, 2, 8, 4, 5, 1, 6, 13, 12, 3...  97.1  0.010299
4  [11, 9, 7, 10, 14, 2, 8, 4, 5, 1, 6, 13, 12, 3...  97.1  0.010299
Percobaan Ke-2
Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [4, 1, 12, 10, 6, 7, 8, 5, 9, 3, 15, 13, 14, 1...  110.3  0.009066
1  [4, 1, 12, 10, 7, 11, 15, 2, 6, 14, 13, 8, 5, ...  110.5  0.009050
2  [15, 6, 1, 10, 14, 12, 11, 5, 7, 9, 3, 8, 4, 1...  114.2  0.008757
3  [4, 1, 12, 10, 7, 11, 15, 8, 9, 3, 2, 6, 13, 5...  118.8  0.008418
4  [13, 14, 4, 12, 7, 15, 11, 3, 1, 6, 8, 5, 10, ...  119.2  0.008389
Generasi Ke-1
Populasi Bertahan :
                                           Kromosome   Cost   F

Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [10, 4, 5, 8, 6, 7, 9, 1, 13, 2, 15, 3, 11, 12...  111.3  0.008985
1  [9, 12, 4, 6, 13, 15, 3, 8, 5, 2, 10, 7, 1, 14...  112.2  0.008913
2  [8, 4, 6, 15, 10, 11, 13, 5, 2, 9, 12, 14, 1, ...  114.8  0.008711
3  [8, 4, 6, 15, 10, 11, 13, 5, 2, 9, 12, 14, 1, ...  114.8  0.008711
4  [10, 3, 12, 11, 14, 6, 9, 15, 13, 7, 1, 4, 5, ...  118.1  0.008467
Generasi Ke-1
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [9, 11, 10, 4, 5, 8, 6, 7, 1, 13, 2, 15, 3, 12...  106.2  0.009416
1  [1, 11, 13, 2, 8, 12, 9, 4, 6, 15, 3, 5, 10, 7...  110.7  0.009033
2  [10, 4, 5, 8, 6, 7, 9, 1, 13, 2, 15, 3, 11, 12...  111.3  0.008985
3  [9, 12, 4, 6, 13, 15, 3, 8, 5, 2, 10, 7, 1, 14...  112.2  0.008913
4  [8, 4, 6, 15, 10, 11, 13, 5, 2, 9, 12, 14, 1, ...  114.8  0.008711
Generasi Ke-2
Populasi Bertahan :
                                           Kromosome   Cos

Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [2, 6, 12, 11, 1, 14, 4, 10, 8, 7, 9, 15, 3, 1...  100.4  0.009960
1  [14, 1, 4, 11, 13, 12, 2, 6, 10, 15, 8, 5, 7, ...  111.8  0.008945
2  [4, 2, 6, 15, 13, 3, 12, 11, 1, 14, 5, 9, 7, 8...  111.9  0.008937
3  [4, 10, 8, 6, 7, 9, 13, 3, 5, 15, 2, 12, 11, 1...  112.8  0.008865
4  [6, 9, 1, 11, 12, 7, 3, 15, 10, 4, 8, 5, 14, 2...  113.9  0.008780
Generasi Ke-1
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [4, 10, 8, 6, 7, 9, 15, 3, 2, 12, 11, 1, 14, 1...   98.3  0.010173
1  [2, 6, 12, 11, 1, 14, 4, 10, 8, 7, 9, 15, 3, 1...  100.4  0.009960
2  [2, 6, 12, 11, 1, 14, 4, 10, 8, 7, 9, 15, 3, 1...  100.4  0.009960
3  [2, 6, 12, 11, 1, 14, 4, 10, 8, 7, 9, 15, 3, 1...  100.4  0.009960
4  [4, 10, 8, 6, 2, 12, 11, 1, 14, 7, 9, 15, 3, 1...  103.6  0.009653
Generasi Ke-2
Populasi Bertahan :
                                           Kromosome   Cos

...

Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [3, 14, 6, 12, 4, 7, 15, 13, 2, 1, 8, 11, 9, 1...  91.6  0.010917
1  [3, 14, 6, 12, 4, 7, 15, 13, 2, 1, 8, 11, 9, 1...  91.6  0.010917
2  [3, 14, 6, 12, 4, 7, 15, 13, 2, 1, 8, 11, 9, 1...  96.7  0.010341
3  [3, 14, 6, 12, 4, 7, 15, 13, 2, 1, 8, 11, 9, 1...  97.1  0.010299
4  [3, 14, 6, 12, 4, 7, 15, 13, 2, 1, 8, 11, 9, 1...  97.1  0.010299
Percobaan Ke-3
Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [10, 15, 2, 12, 7, 4, 5, 6, 14, 13, 11, 1, 9, ...  113.9  0.008780
1  [10, 15, 2, 12, 7, 4, 5, 6, 14, 13, 11, 1, 9, ...  113.9  0.008780
2  [14, 6, 15, 5, 9, 10, 1, 13, 4, 8, 3, 11, 7, 2...  115.4  0.008666
3  [14, 6, 15, 5, 9, 10, 1, 13, 4, 8, 3, 11, 7, 2...  115.4  0.008666
4  [14, 6, 15, 5, 9, 10, 1, 13, 4, 8, 3, 11, 7, 2...  118.0  0.008475
Generasi Ke-1
Populasi Bertahan :
                                           Kromosome   Cost   F

                                           Kromosome  Cost   Fitness
0  [11, 15, 2, 8, 14, 13, 6, 9, 3, 1, 4, 10, 7, 1...  98.5  0.010152
1  [11, 15, 2, 8, 14, 13, 6, 9, 3, 1, 4, 10, 7, 1...  98.5  0.010152
2  [11, 15, 2, 8, 14, 13, 6, 9, 3, 1, 4, 10, 7, 1...  98.5  0.010152
3  [11, 15, 2, 8, 14, 13, 6, 9, 3, 1, 4, 10, 7, 1...  98.5  0.010152
4  [11, 15, 2, 8, 14, 13, 6, 9, 3, 1, 4, 10, 7, 1...  98.5  0.010152
Generasi Ke-10
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [13, 11, 2, 3, 1, 15, 4, 10, 8, 14, 12, 7, 9, ...  98.5  0.010152
1  [13, 11, 2, 3, 1, 15, 4, 10, 8, 14, 12, 7, 9, ...  98.5  0.010152
2  [13, 11, 2, 3, 1, 15, 4, 10, 8, 14, 12, 7, 9, ...  98.5  0.010152
3  [13, 11, 2, 3, 1, 15, 4, 10, 8, 14, 12, 7, 9, ...  98.5  0.010152
4  [13, 11, 2, 3, 1, 15, 4, 10, 8, 14, 12, 7, 9, ...  98.5  0.010152
Generasi Ke-11
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [12, 11, 3, 2, 14, 15, 7, 10, 

                                           Kromosome   Cost   Fitness
0  [15, 14, 7, 3, 1, 12, 11, 9, 2, 13, 10, 8, 6, ...  102.2  0.009785
1  [15, 14, 7, 3, 1, 12, 11, 9, 2, 13, 10, 8, 6, ...  108.9  0.009183
2  [15, 14, 7, 3, 1, 12, 11, 9, 2, 13, 10, 8, 6, ...  108.9  0.009183
3  [15, 14, 7, 3, 1, 12, 11, 9, 2, 13, 10, 8, 6, ...  108.9  0.009183
4  [15, 14, 7, 3, 1, 12, 11, 9, 2, 13, 10, 8, 6, ...  108.9  0.009183
Generasi Ke-19
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [7, 13, 15, 2, 1, 12, 10, 5, 8, 3, 11, 14, 6, ...  102.2  0.009785
1  [7, 13, 15, 2, 1, 12, 10, 5, 8, 3, 11, 14, 6, ...  108.9  0.009183
2  [7, 13, 15, 2, 1, 12, 10, 5, 8, 3, 11, 14, 6, ...  108.9  0.009183
3  [7, 13, 15, 2, 1, 12, 10, 5, 8, 3, 11, 14, 6, ...  108.9  0.009183
4  [7, 13, 15, 2, 1, 12, 10, 5, 8, 3, 11, 14, 6, ...  108.9  0.009183
Percobaan Ke-6
Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [8,

                                           Kromosome   Cost   Fitness
0  [12, 14, 13, 9, 8, 7, 15, 2, 5, 11, 1, 10, 6, ...  101.5  0.009852
1  [12, 14, 13, 9, 8, 7, 15, 2, 5, 11, 1, 10, 6, ...  109.3  0.009149
2  [12, 14, 13, 9, 8, 7, 15, 2, 5, 11, 1, 10, 6, ...  109.3  0.009149
3  [12, 14, 13, 9, 8, 7, 15, 2, 5, 11, 1, 10, 6, ...  109.3  0.009149
4  [12, 13, 14, 9, 8, 7, 15, 2, 5, 4, 6, 10, 1, 1...  109.3  0.009149
Generasi Ke-4
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [12, 14, 4, 5, 8, 7, 15, 2, 9, 11, 1, 10, 6, 1...  101.5  0.009852
1  [12, 14, 4, 5, 8, 7, 15, 2, 9, 11, 1, 10, 6, 1...  109.3  0.009149
2  [12, 14, 4, 5, 8, 7, 15, 2, 9, 11, 1, 10, 6, 1...  109.3  0.009149
3  [12, 14, 4, 5, 8, 7, 15, 2, 9, 11, 1, 10, 6, 1...  109.3  0.009149
4  [12, 4, 14, 9, 8, 15, 7, 2, 5, 13, 6, 10, 1, 1...  109.3  0.009149
Generasi Ke-5
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [9, 14, 4, 15, 8, 5

                                           Kromosome   Cost   Fitness
0  [11, 2, 4, 3, 6, 8, 15, 9, 13, 14, 12, 1, 7, 5...   99.5  0.010050
1  [11, 2, 4, 3, 6, 8, 15, 9, 13, 14, 12, 1, 7, 5...   99.5  0.010050
2  [11, 12, 14, 1, 9, 3, 8, 13, 2, 7, 6, 15, 4, 1...  101.9  0.009814
3  [11, 12, 14, 1, 9, 3, 8, 13, 2, 7, 6, 15, 4, 1...  102.0  0.009804
4  [11, 12, 14, 1, 9, 3, 8, 13, 2, 7, 6, 15, 4, 1...  102.0  0.009804
Generasi Ke-16
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [11, 2, 4, 14, 6, 1, 15, 9, 13, 3, 12, 8, 7, 5...   99.5  0.010050
1  [11, 2, 4, 14, 6, 1, 15, 9, 13, 3, 12, 8, 7, 5...   99.5  0.010050
2  [15, 14, 12, 1, 9, 8, 3, 13, 4, 7, 6, 11, 2, 1...  101.9  0.009814
3  [15, 14, 12, 1, 9, 8, 3, 13, 4, 7, 6, 11, 2, 1...  102.0  0.009804
4  [15, 14, 12, 1, 9, 8, 3, 13, 4, 7, 6, 11, 2, 1...  102.0  0.009804
Generasi Ke-17
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [11, 2, 4, 14, 6,

# Pengujian Jumlah Populasi

In [6]:
jenis_populasi = [50,60,70,80,90,100]
for i in jenis_populasi:
    print('Jumlah Populasi : {}'.format(i))
    mtsp = MTSP(df_node,0,[(0,7),(8,14)],populasi=i,generasi=20)
    for loop in range(10):
        print('Percobaan ke-{}'.format(loop))
        mtsp.main(0.9,0.1)

Jumlah Populasi : 50
Percobaan ke-0
Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [2, 14, 4, 7, 11, 15, 3, 12, 10, 1, 6, 13, 9, ...  102.8  0.009728
1  [8, 11, 12, 15, 6, 2, 9, 10, 7, 3, 5, 13, 1, 1...  105.8  0.009452
2  [14, 10, 13, 9, 4, 11, 6, 1, 5, 15, 12, 3, 7, ...  107.0  0.009346
3  [10, 3, 12, 14, 7, 8, 13, 1, 6, 4, 15, 11, 5, ...  109.0  0.009174
4  [1, 3, 2, 5, 6, 14, 13, 10, 4, 15, 12, 11, 7, ...  109.2  0.009158
Generasi Ke-1
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [13, 5, 2, 12, 15, 9, 8, 7, 4, 6, 11, 3, 14, 1...   94.4  0.010593
1  [14, 10, 13, 9, 4, 6, 11, 1, 5, 15, 12, 3, 7, ...  100.0  0.010000
2  [14, 9, 13, 7, 3, 15, 5, 2, 4, 6, 12, 11, 8, 1...  102.4  0.009766
3  [2, 14, 4, 7, 11, 15, 3, 12, 10, 1, 6, 13, 9, ...  102.8  0.009728
4  [15, 6, 4, 3, 12, 7, 10, 1, 13, 14, 11, 9, 8, ...  105.0  0.009524
Generasi Ke-2
Populasi Bertahan :
                      

Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [2, 12, 11, 6, 14, 10, 1, 8, 3, 15, 9, 7, 5, 4...  101.3  0.009872
1  [6, 4, 14, 9, 11, 15, 3, 1, 13, 7, 12, 8, 10, ...  102.2  0.009785
2  [4, 13, 5, 14, 8, 9, 7, 2, 10, 1, 15, 11, 3, 1...  103.3  0.009681
3  [6, 4, 14, 9, 11, 15, 3, 1, 2, 12, 10, 5, 8, 1...  104.0  0.009615
4  [6, 4, 14, 9, 11, 15, 3, 1, 2, 12, 10, 5, 8, 1...  104.0  0.009615
Generasi Ke-1
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [5, 8, 12, 2, 10, 11, 13, 1, 6, 9, 3, 15, 7, 1...  100.4  0.009960
1  [2, 12, 11, 6, 14, 10, 1, 8, 3, 15, 9, 7, 5, 4...  101.3  0.009872
2  [2, 12, 11, 6, 14, 10, 1, 8, 3, 15, 9, 7, 5, 4...  101.3  0.009872
3  [13, 11, 5, 12, 3, 8, 15, 1, 4, 6, 10, 14, 9, ...  101.5  0.009852
4  [13, 11, 5, 12, 3, 8, 15, 1, 4, 6, 10, 14, 9, ...  101.5  0.009852
Generasi Ke-2
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0 

Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [8, 13, 9, 11, 3, 15, 14, 7, 6, 5, 12, 2, 1, 1...  100.9  0.009911
1  [15, 9, 11, 1, 10, 13, 6, 4, 14, 12, 2, 7, 3, ...  101.8  0.009823
2  [15, 9, 11, 1, 10, 13, 6, 4, 5, 14, 8, 12, 2, ...  104.1  0.009606
3  [15, 9, 11, 1, 10, 13, 6, 4, 5, 3, 8, 14, 2, 1...  104.9  0.009533
4  [15, 9, 11, 1, 10, 13, 6, 4, 12, 2, 3, 5, 7, 8...  105.3  0.009497
Generasi Ke-2
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [8, 13, 9, 11, 3, 15, 14, 2, 6, 5, 12, 7, 1, 1...   95.8  0.010438
1  [8, 13, 9, 11, 3, 15, 14, 2, 6, 5, 12, 7, 1, 1...  100.9  0.009911
2  [8, 13, 9, 11, 3, 15, 14, 7, 6, 5, 12, 2, 1, 1...  100.9  0.009911
3  [15, 9, 11, 1, 10, 13, 6, 4, 14, 12, 2, 3, 7, ...  101.1  0.009891
4  [5, 1, 10, 2, 15, 3, 8, 14, 12, 7, 9, 11, 4, 1...  101.6  0.009843
Generasi Ke-3
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  

Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [6, 8, 9, 7, 2, 11, 1, 10, 4, 13, 15, 3, 5, 12...  103.2  0.009690
1  [10, 14, 13, 2, 3, 8, 11, 9, 6, 5, 4, 1, 7, 15...  105.4  0.009488
2  [10, 14, 13, 2, 3, 8, 11, 9, 6, 5, 4, 1, 7, 15...  105.4  0.009488
3  [13, 1, 4, 15, 7, 9, 3, 5, 6, 2, 11, 12, 8, 10...  108.4  0.009225
4  [8, 7, 2, 6, 13, 4, 3, 15, 11, 12, 9, 1, 10, 1...  108.9  0.009183
Generasi Ke-1
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [6, 8, 9, 7, 2, 11, 1, 10, 4, 13, 15, 3, 5, 12...  103.2  0.009690
1  [6, 8, 9, 7, 2, 11, 1, 10, 4, 13, 15, 3, 5, 12...  103.2  0.009690
2  [2, 3, 8, 9, 13, 14, 10, 4, 6, 11, 12, 15, 1, ...  104.2  0.009597
3  [10, 14, 13, 2, 3, 8, 11, 9, 6, 5, 4, 1, 7, 15...  105.4  0.009488
4  [10, 14, 13, 2, 3, 8, 11, 9, 6, 5, 4, 1, 7, 15...  105.4  0.009488
Generasi Ke-2
Populasi Bertahan :
                                           Kromosome   Cos

4  [4, 1, 10, 14, 13, 5, 8, 6, 2, 7, 9, 11, 15, 3...  76.2  0.013123
Generasi Ke-19
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [4, 10, 1, 14, 13, 8, 5, 6, 2, 7, 9, 11, 15, 3...  74.7  0.013387
1  [4, 10, 1, 14, 13, 5, 8, 6, 2, 7, 9, 11, 15, 3...  74.9  0.013351
2  [4, 10, 1, 14, 13, 5, 8, 6, 2, 7, 9, 11, 15, 3...  74.9  0.013351
3  [4, 10, 1, 14, 13, 5, 8, 6, 2, 7, 9, 11, 15, 3...  74.9  0.013351
4  [4, 10, 1, 14, 13, 5, 8, 6, 2, 7, 9, 11, 15, 3...  74.9  0.013351
Percobaan ke-4
Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [1, 8, 7, 13, 11, 12, 10, 14, 6, 4, 5, 2, 9, 1...  100.6  0.009940
1  [11, 7, 13, 14, 15, 12, 3, 4, 9, 10, 1, 5, 8, ...  105.6  0.009470
2  [4, 7, 3, 15, 11, 6, 10, 14, 8, 9, 13, 12, 5, ...  108.9  0.009183
3  [4, 1, 9, 5, 7, 10, 12, 15, 2, 8, 3, 11, 13, 1...  109.0  0.009174
4  [1, 6, 12, 3, 9, 15, 8, 11, 4, 10, 2, 7, 14, 1...  110.0  0.009091
Generasi Ke-1

...

Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [4, 14, 1, 7, 12, 15, 3, 8, 13, 5, 2, 11, 9, 1...  82.1  0.012180
1  [4, 6, 15, 12, 3, 9, 13, 5, 2, 7, 11, 8, 1, 14...  84.5  0.011834
2  [4, 6, 13, 5, 2, 7, 3, 15, 8, 11, 12, 9, 1, 10...  85.8  0.011655
3  [4, 6, 15, 12, 3, 13, 5, 2, 7, 8, 11, 9, 1, 10...  85.8  0.011655
4  [4, 6, 15, 12, 3, 9, 11, 5, 1, 10, 13, 2, 8, 7...  87.3  0.011455
Generasi Ke-6
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [4, 14, 1, 7, 12, 15, 3, 8, 13, 5, 2, 11, 9, 1...  82.1  0.012180
1  [13, 5, 2, 7, 3, 15, 12, 4, 6, 8, 11, 9, 1, 10...  83.8  0.011933
2  [4, 6, 15, 12, 3, 9, 13, 5, 2, 7, 11, 8, 1, 14...  84.5  0.011834
3  [4, 6, 15, 12, 3, 9, 13, 5, 2, 7, 11, 8, 1, 14...  84.5  0.011834
4  [4, 6, 15, 12, 3, 9, 13, 5, 2, 7, 11, 8, 1, 14...  84.5  0.011834
Generasi Ke-7
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [4, 6, 15, 1

Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [6, 1, 10, 13, 7, 11, 9, 4, 5, 14, 2, 12, 3, 1...  91.3  0.010953
1  [2, 5, 7, 13, 9, 11, 1, 6, 10, 15, 3, 12, 8, 1...  91.8  0.010893
2  [8, 14, 6, 1, 10, 15, 3, 12, 2, 5, 13, 9, 11, ...  93.3  0.010718
3  [6, 1, 10, 7, 11, 15, 3, 13, 5, 9, 12, 2, 8, 1...  94.0  0.010638
4  [6, 1, 10, 7, 11, 15, 3, 13, 5, 9, 12, 2, 8, 1...  94.0  0.010638
Generasi Ke-8
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [6, 1, 10, 7, 15, 3, 8, 4, 14, 11, 12, 9, 2, 5...  89.5  0.011173
1  [6, 1, 10, 13, 7, 11, 9, 4, 5, 14, 2, 12, 3, 1...  91.3  0.010953
2  [2, 5, 7, 13, 9, 11, 1, 6, 10, 15, 3, 12, 8, 1...  91.8  0.010893
3  [8, 14, 6, 1, 10, 15, 3, 12, 2, 5, 13, 9, 11, ...  93.3  0.010718
4  [6, 1, 10, 7, 11, 15, 3, 13, 5, 9, 12, 2, 8, 1...  94.0  0.010638
Generasi Ke-9
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [6, 1, 10, 7

Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [4, 6, 13, 9, 1, 14, 10, 11, 8, 5, 2, 7, 12, 3...  84.6  0.011820
1  [6, 15, 12, 3, 9, 10, 14, 1, 4, 5, 8, 11, 13, ...  84.9  0.011779
2  [6, 15, 12, 3, 9, 10, 14, 1, 4, 5, 8, 11, 13, ...  84.9  0.011779
3  [6, 15, 12, 3, 9, 10, 14, 1, 4, 5, 8, 11, 7, 2...  85.1  0.011751
4  [4, 6, 13, 9, 1, 14, 10, 11, 8, 5, 2, 7, 12, 3...  87.3  0.011455
Generasi Ke-8
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [4, 7, 13, 9, 1, 14, 10, 11, 8, 5, 2, 6, 12, 3...  84.6  0.011820
1  [6, 15, 12, 3, 9, 10, 14, 1, 4, 5, 8, 11, 13, ...  84.9  0.011779
2  [6, 15, 12, 3, 9, 10, 14, 1, 4, 5, 8, 11, 13, ...  84.9  0.011779
3  [6, 15, 12, 3, 9, 10, 14, 1, 4, 5, 8, 11, 7, 2...  85.1  0.011751
4  [6, 15, 12, 3, 9, 10, 14, 1, 4, 5, 11, 7, 8, 1...  85.2  0.011737
Generasi Ke-9
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [4, 6, 13, 1

Populasi Bertahan :
                                           Kromosome  Cost  Fitness
0  [6, 1, 10, 7, 15, 12, 3, 2, 13, 5, 8, 9, 11, 1...  81.3   0.0123
1  [6, 1, 10, 7, 15, 12, 3, 2, 13, 5, 8, 9, 11, 1...  81.3   0.0123
2  [6, 1, 10, 7, 15, 12, 3, 2, 13, 5, 8, 9, 11, 1...  81.3   0.0123
3  [6, 1, 10, 7, 15, 12, 3, 2, 13, 5, 8, 9, 11, 1...  81.3   0.0123
4  [6, 1, 10, 7, 15, 12, 3, 2, 13, 5, 8, 9, 11, 1...  81.3   0.0123
Generasi Ke-11
Populasi Bertahan :
                                           Kromosome  Cost  Fitness
0  [6, 1, 10, 7, 15, 12, 3, 2, 13, 5, 8, 9, 11, 1...  81.3   0.0123
1  [6, 1, 10, 7, 15, 12, 3, 2, 13, 5, 8, 9, 11, 1...  81.3   0.0123
2  [6, 1, 10, 7, 15, 12, 3, 2, 13, 5, 8, 9, 11, 1...  81.3   0.0123
3  [11, 1, 10, 7, 15, 12, 3, 2, 13, 5, 8, 9, 6, 1...  81.3   0.0123
4  [6, 1, 10, 7, 15, 12, 3, 2, 13, 5, 8, 9, 11, 1...  81.3   0.0123
Generasi Ke-12
Populasi Bertahan :
                                           Kromosome  Cost  Fitness
0  [6, 1, 10, 7, 15, 12, 3

Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [6, 4, 1, 14, 13, 7, 8, 5, 10, 11, 9, 2, 15, 3...  81.4  0.012285
1  [12, 3, 11, 9, 10, 1, 14, 4, 5, 8, 15, 2, 7, 1...  82.2  0.012165
2  [12, 3, 11, 9, 10, 1, 14, 4, 5, 8, 7, 2, 15, 1...  82.2  0.012165
3  [12, 3, 11, 9, 10, 1, 14, 4, 5, 8, 7, 2, 15, 1...  82.9  0.012063
4  [5, 12, 3, 11, 9, 10, 1, 14, 4, 8, 15, 2, 7, 1...  84.5  0.011834
Generasi Ke-13
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [6, 4, 1, 14, 13, 7, 8, 5, 10, 11, 9, 2, 15, 3...  81.4  0.012285
1  [12, 7, 11, 9, 10, 1, 14, 4, 5, 8, 15, 2, 3, 1...  82.2  0.012165
2  [12, 3, 11, 9, 10, 1, 14, 4, 5, 8, 7, 2, 15, 1...  82.2  0.012165
3  [12, 3, 11, 9, 10, 1, 14, 4, 5, 8, 15, 2, 7, 1...  82.2  0.012165
4  [12, 3, 11, 9, 10, 1, 14, 4, 5, 8, 15, 2, 7, 1...  82.2  0.012165
Generasi Ke-14
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [6, 4, 1, 

# Pengujian Jumlah Kurir

In [7]:
salesman = [[(0,14)],[(0,7),(8,14)],[(0,4),(5,9),(10,14)],[(0,3),(4,7),(8,11),(12,14)],[(0,2),(3,5),(6,8),(9,11),(12,14)],
            [(0,1),(2,4),(5,8),(9,10),(11,12),(13,14)],[(0,1),(2,4),(5,6),(7,8),(9,10),(11,12),(13,14)],
            [(0,1),(2,3),(4,5),(6,7),(8,9),(10,11),(12,13),(14,14)],[(0,0),(1,2),(3,3),(4,5),(6,7),(8,9),(10,11),(12,13),(14,14)],
            [(0,0),(1,2),(3,3),(4,4),(5,5),(6,7),(8,9),(10,11),(12,13),(14,14)]]
for i in salesman:
    print('Jumlah Salesman : {}'.format(len(i)))
    mtsp = MTSP(df_node,0,i,populasi=70,generasi=20)
    for loop in range(10):
        print('Percobaan ke-{}'.format(loop))
        mtsp.main(0.9,0.1)

Jumlah Salesman : 1
Percobaan ke-0
Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [13, 8, 12, 14, 10, 1, 4, 5, 2, 3, 7, 15, 11, ...   95.8  0.010438
1  [4, 9, 3, 11, 10, 13, 12, 15, 5, 6, 8, 2, 7, 1...   97.0  0.010309
2  [14, 1, 13, 8, 10, 15, 12, 9, 3, 11, 7, 4, 5, ...   97.4  0.010267
3  [7, 13, 5, 12, 3, 14, 10, 1, 4, 6, 9, 15, 2, 1...   98.5  0.010152
4  [2, 12, 7, 4, 6, 10, 1, 15, 3, 8, 9, 13, 5, 14...  101.2  0.009881
Generasi Ke-1
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [4, 13, 8, 1, 10, 11, 2, 12, 14, 9, 7, 15, 3, ...  93.0  0.010753
1  [14, 1, 10, 11, 13, 9, 12, 8, 4, 6, 5, 7, 3, 2...  94.3  0.010604
2  [13, 8, 12, 14, 10, 1, 4, 5, 2, 3, 7, 15, 11, ...  95.8  0.010438
3  [4, 9, 3, 11, 10, 13, 12, 15, 5, 6, 8, 2, 7, 1...  97.0  0.010309
4  [14, 1, 13, 8, 10, 15, 12, 9, 3, 11, 7, 4, 5, ...  97.4  0.010267
Generasi Ke-2
Populasi Bertahan :
                             

Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [13, 10, 14, 1, 6, 11, 9, 8, 12, 4, 5, 7, 3, 1...  67.0  0.014925
1  [4, 6, 14, 1, 10, 11, 15, 12, 8, 13, 5, 7, 3, ...  67.0  0.014925
2  [4, 15, 13, 14, 10, 1, 11, 9, 12, 8, 5, 7, 6, ...  71.1  0.014065
3  [14, 1, 10, 11, 9, 13, 5, 7, 8, 12, 15, 3, 2, ...  71.5  0.013986
4  [14, 1, 10, 11, 9, 13, 5, 7, 8, 12, 15, 3, 2, ...  71.5  0.013986
Percobaan ke-1
Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [2, 6, 14, 1, 8, 11, 10, 4, 7, 5, 13, 9, 12, 3...   92.3  0.010834
1  [6, 4, 13, 8, 9, 1, 3, 15, 12, 11, 14, 5, 10, ...   98.1  0.010194
2  [4, 1, 14, 5, 9, 2, 13, 6, 10, 7, 8, 3, 11, 15...  100.3  0.009970
3  [4, 3, 11, 8, 13, 5, 6, 1, 14, 12, 7, 9, 10, 1...  101.4  0.009862
4  [12, 14, 5, 13, 2, 8, 9, 15, 3, 10, 7, 11, 1, ...  101.6  0.009843
Generasi Ke-1
Populasi Bertahan :
                                           Kromosome  Cost   Fi

                                           Kromosome  Cost   Fitness
0  [4, 6, 1, 14, 10, 8, 2, 5, 13, 9, 11, 3, 15, 1...  69.2  0.014451
1  [4, 6, 1, 14, 10, 8, 2, 5, 13, 9, 11, 3, 15, 1...  69.2  0.014451
2  [6, 4, 1, 14, 10, 13, 8, 2, 5, 9, 11, 12, 3, 1...  69.5  0.014388
3  [4, 6, 2, 5, 13, 8, 1, 14, 12, 9, 11, 10, 3, 1...  69.6  0.014368
4  [4, 6, 2, 1, 13, 8, 5, 14, 10, 9, 3, 12, 11, 1...  69.6  0.014368
Percobaan ke-2
Generasi Ke-0
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [6, 4, 14, 1, 7, 8, 5, 9, 10, 3, 15, 12, 11, 2...  88.3  0.011325
1  [8, 13, 1, 10, 14, 12, 3, 15, 7, 5, 11, 2, 6, ...  94.8  0.010549
2  [15, 13, 10, 14, 1, 5, 7, 6, 8, 2, 3, 12, 11, ...  95.8  0.010438
3  [12, 10, 6, 4, 14, 1, 7, 8, 5, 9, 3, 15, 11, 2...  96.6  0.010352
4  [14, 4, 12, 15, 9, 3, 13, 8, 2, 6, 5, 7, 11, 1...  96.7  0.010341
Generasi Ke-1
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [6, 4, 14, 1, 7, 

Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [6, 13, 8, 5, 10, 14, 1, 11, 15, 3, 12, 9, 7, ...  70.7  0.014144
1  [5, 8, 13, 1, 10, 14, 12, 3, 15, 7, 11, 9, 2, ...  71.2  0.014045
2  [5, 13, 1, 10, 14, 6, 8, 12, 3, 15, 7, 11, 9, ...  72.2  0.013850
3  [5, 13, 1, 10, 14, 6, 8, 12, 3, 15, 7, 11, 2, ...  72.2  0.013850
4  [5, 13, 1, 10, 14, 6, 8, 12, 3, 15, 7, 11, 9, ...  72.2  0.013850
Generasi Ke-19
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [6, 13, 8, 5, 10, 14, 1, 11, 15, 3, 12, 9, 7, ...  70.7  0.014144
1  [5, 8, 13, 1, 10, 14, 12, 3, 15, 7, 11, 9, 2, ...  71.2  0.014045
2  [6, 5, 13, 1, 10, 14, 8, 12, 3, 15, 7, 11, 9, ...  71.2  0.014045
3  [5, 8, 13, 1, 10, 14, 12, 3, 15, 7, 11, 9, 2, ...  71.2  0.014045
4  [6, 8, 5, 13, 1, 10, 14, 11, 12, 3, 15, 7, 9, ...  71.4  0.014006
Percobaan ke-3
Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitnes

Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [6, 4, 2, 5, 10, 1, 14, 11, 9, 3, 15, 12, 7, 8...  68.8  0.014535
1  [6, 4, 2, 5, 10, 1, 14, 11, 9, 3, 15, 12, 7, 8...  68.8  0.014535
2  [6, 4, 2, 5, 10, 1, 14, 11, 9, 3, 15, 12, 7, 8...  68.8  0.014535
3  [6, 4, 2, 5, 10, 1, 14, 11, 9, 3, 15, 12, 7, 8...  68.8  0.014535
4  [6, 4, 2, 5, 10, 1, 14, 11, 9, 3, 15, 12, 7, 8...  68.8  0.014535
Generasi Ke-19
Populasi Bertahan :
                                           Kromosome  Cost   Fitness
0  [6, 4, 2, 5, 10, 1, 14, 11, 9, 12, 3, 15, 7, 8...  67.9  0.014728
1  [6, 4, 2, 5, 10, 1, 14, 11, 9, 3, 15, 12, 7, 8...  68.8  0.014535
2  [6, 4, 2, 5, 10, 15, 14, 11, 9, 3, 1, 12, 7, 8...  68.8  0.014535
3  [6, 4, 2, 5, 10, 1, 14, 11, 9, 3, 15, 12, 7, 8...  68.8  0.014535
4  [6, 4, 2, 5, 10, 1, 14, 11, 9, 3, 15, 12, 7, 8...  68.8  0.014535
Percobaan ke-4
Generasi Ke-0
Populasi Bertahan :
                                           Kromosome  Cost   Fitness

...

Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [2, 10, 14, 4, 6, 1, 5, 15, 7, 8, 3, 12, 11, 9...  151.2  0.006614
1  [6, 10, 14, 1, 5, 4, 11, 15, 2, 8, 3, 12, 9, 7...  151.7  0.006592
2  [2, 10, 14, 6, 4, 5, 15, 1, 7, 8, 9, 11, 3, 12...  152.2  0.006570
3  [2, 10, 14, 6, 4, 5, 15, 1, 11, 8, 9, 7, 3, 12...  152.2  0.006570
4  [2, 10, 14, 4, 6, 5, 15, 1, 7, 8, 9, 11, 3, 12...  152.2  0.006570
Generasi Ke-12
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [2, 10, 15, 6, 4, 5, 14, 1, 7, 8, 9, 11, 3, 12...  150.2  0.006658
1  [2, 10, 14, 4, 6, 12, 5, 15, 7, 8, 3, 1, 11, 9...  151.2  0.006614
2  [2, 10, 14, 4, 6, 1, 5, 15, 7, 8, 3, 12, 11, 9...  151.2  0.006614
3  [6, 10, 14, 1, 5, 4, 11, 15, 2, 8, 3, 12, 9, 7...  151.7  0.006592
4  [6, 10, 14, 1, 5, 4, 2, 15, 11, 8, 3, 12, 9, 7...  152.0  0.006579
Generasi Ke-13
Populasi Bertahan :
                                           Kromosome   Cost   Fitness


Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [10, 12, 7, 6, 13, 4, 11, 9, 3, 15, 5, 8, 14, ...  144.1  0.006940
1  [10, 12, 7, 6, 13, 4, 11, 9, 3, 15, 5, 8, 14, ...  144.1  0.006940
2  [10, 3, 15, 4, 1, 14, 13, 5, 2, 8, 11, 7, 12, ...  150.9  0.006627
3  [8, 3, 15, 4, 10, 14, 13, 5, 7, 6, 9, 11, 12, ...  152.0  0.006579
4  [8, 3, 15, 4, 10, 14, 13, 5, 1, 6, 9, 11, 12, ...  152.0  0.006579
Generasi Ke-13
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [10, 12, 7, 6, 13, 4, 11, 9, 3, 15, 5, 8, 14, ...  144.1  0.006940
1  [10, 12, 7, 6, 13, 4, 11, 9, 3, 15, 5, 8, 14, ...  144.1  0.006940
2  [10, 3, 15, 4, 1, 14, 13, 5, 2, 8, 11, 7, 12, ...  150.9  0.006627
3  [10, 11, 7, 6, 13, 4, 12, 3, 1, 14, 5, 8, 15, ...  151.0  0.006623
4  [8, 3, 15, 4, 10, 14, 13, 5, 7, 6, 9, 11, 12, ...  152.0  0.006579
Generasi Ke-14
Populasi Bertahan :
                                           Kromosome   Cost   Fitness


Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [14, 5, 2, 10, 4, 1, 9, 11, 7, 12, 15, 3, 13, ...  149.1  0.006707
1  [6, 11, 10, 5, 2, 3, 8, 13, 14, 1, 9, 7, 12, 1...  152.1  0.006575
2  [2, 12, 13, 14, 6, 8, 3, 15, 9, 7, 10, 1, 5, 1...  152.8  0.006545
3  [2, 12, 13, 14, 6, 8, 3, 15, 9, 7, 10, 1, 5, 1...  152.8  0.006545
4  [2, 12, 13, 14, 6, 8, 3, 15, 9, 7, 10, 1, 5, 1...  152.8  0.006545
Generasi Ke-14
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [14, 5, 2, 10, 4, 1, 9, 11, 7, 12, 15, 3, 13, ...  149.1  0.006707
1  [6, 11, 10, 5, 2, 3, 8, 13, 14, 1, 9, 7, 12, 1...  152.1  0.006575
2  [6, 11, 10, 2, 5, 3, 8, 13, 1, 14, 12, 15, 7, ...  152.1  0.006575
3  [14, 8, 2, 10, 5, 13, 15, 3, 12, 7, 6, 1, 9, 1...  152.6  0.006553
4  [2, 12, 13, 14, 6, 8, 3, 15, 9, 7, 10, 1, 5, 1...  152.8  0.006545
Generasi Ke-15
Populasi Bertahan :
                                           Kromosome   Cost   Fitness


4  [6, 9, 8, 13, 4, 5, 15, 3, 11, 10, 14, 1, 7, 1...  148.7  0.006725
Generasi Ke-13
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [6, 9, 8, 13, 4, 5, 15, 3, 11, 10, 7, 12, 1, 1...  148.7  0.006725
1  [6, 9, 8, 13, 4, 5, 15, 3, 11, 10, 7, 12, 1, 1...  148.7  0.006725
2  [1, 9, 8, 13, 4, 5, 15, 3, 11, 10, 7, 12, 6, 1...  148.7  0.006725
3  [6, 9, 8, 13, 4, 5, 15, 3, 11, 10, 14, 1, 7, 1...  148.7  0.006725
4  [6, 9, 8, 13, 4, 11, 15, 3, 5, 10, 14, 1, 7, 1...  148.7  0.006725
Generasi Ke-14
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [6, 9, 8, 13, 4, 5, 15, 3, 11, 10, 7, 12, 1, 1...  148.7  0.006725
1  [6, 9, 8, 13, 4, 5, 15, 3, 11, 10, 7, 12, 1, 1...  148.7  0.006725
2  [1, 9, 8, 13, 4, 5, 15, 3, 11, 10, 7, 12, 6, 1...  148.7  0.006725
3  [6, 9, 8, 13, 4, 5, 15, 3, 11, 10, 14, 1, 7, 1...  148.7  0.006725
4  [6, 8, 9, 13, 4, 11, 15, 3, 5, 10, 14, 1, 7, 1...  148.7  0.006725
Generasi Ke-15
Popul

Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [5, 11, 9, 2, 4, 6, 7, 12, 3, 15, 1, 14, 10, 1...  144.6  0.006916
1  [5, 11, 9, 6, 4, 8, 7, 12, 14, 10, 13, 2, 15, ...  145.9  0.006854
2  [5, 11, 9, 6, 4, 8, 7, 12, 14, 10, 13, 2, 3, 1...  145.9  0.006854
3  [5, 11, 9, 2, 4, 6, 7, 12, 3, 15, 1, 14, 8, 10...  146.3  0.006835
4  [5, 11, 9, 6, 4, 2, 10, 8, 7, 12, 3, 15, 1, 14...  146.3  0.006835
Generasi Ke-15
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [5, 11, 9, 2, 4, 6, 7, 12, 3, 15, 1, 14, 10, 1...  144.6  0.006916
1  [5, 11, 9, 6, 4, 8, 7, 12, 14, 10, 13, 2, 15, ...  145.9  0.006854
2  [5, 11, 9, 6, 4, 8, 7, 12, 14, 10, 13, 2, 3, 1...  145.9  0.006854
3  [5, 11, 9, 6, 4, 8, 7, 12, 14, 10, 13, 2, 3, 1...  145.9  0.006854
4  [5, 11, 9, 2, 4, 6, 7, 12, 3, 15, 1, 14, 8, 10...  146.3  0.006835
Generasi Ke-16
Populasi Bertahan :
                                           Kromosome   Cost   Fitness


# Main

In [8]:
awal = 0
kurir = [(0,14)]

mtsp = MTSP(df_node,awal,kurir,populasi=70,generasi=20)
mtsp.main(0.9,0.1)
df_final = mtsp.df_populasi

Generasi Ke-0
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [12, 5, 15, 3, 7, 2, 6, 9, 11, 8, 10, 1, 13, 1...   98.5  0.010152
1  [2, 4, 6, 1, 9, 13, 7, 3, 11, 15, 12, 14, 10, ...  100.4  0.009960
2  [6, 11, 2, 13, 14, 10, 15, 3, 9, 1, 8, 4, 5, 1...  101.4  0.009862
3  [15, 2, 10, 1, 12, 9, 3, 13, 7, 11, 5, 8, 6, 4...  101.8  0.009823
4  [15, 2, 10, 1, 12, 9, 3, 13, 7, 11, 5, 8, 6, 4...  101.8  0.009823
Generasi Ke-1
Populasi Bertahan :
                                           Kromosome   Cost   Fitness
0  [15, 3, 7, 11, 6, 14, 4, 13, 5, 8, 12, 1, 10, ...   98.3  0.010173
1  [6, 4, 3, 15, 2, 10, 1, 12, 9, 13, 7, 11, 5, 8...   98.3  0.010173
2  [12, 15, 2, 10, 1, 9, 3, 13, 7, 11, 5, 8, 6, 4...   98.4  0.010163
3  [12, 5, 15, 3, 7, 2, 6, 9, 11, 8, 10, 1, 13, 1...   98.5  0.010152
4  [2, 4, 6, 1, 9, 13, 7, 3, 11, 15, 12, 14, 10, ...  100.4  0.009960
Generasi Ke-2
Populasi Bertahan :
                                           Kromosome  Cost

# Mapping

In [9]:
hasil_mapping = []
jarak = [[awal]+df_final.loc[0,'Kromosome'][s:e+1]+[awal] for s,e in kurir]
jarak = [j for i in jarak for j in i]

for i in jarak:
    hasil_mapping.append(mapping[i])
hasil_mapping

['Dinoyo',
 'Lowokwaru',
 'Jatimulyo',
 'Sawojajar',
 'Lesanpuro',
 'Samaan',
 'Polowijen',
 'Purwodadi',
 'Arjosari',
 'Jodipan ',
 'Wonokoyo',
 'Bumiayu',
 'Kebonsari',
 'Bandulan',
 'Klojen',
 'Oro-Oro Dowo',
 'Dinoyo']